In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio,pytorch_lightning
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

def _IMPORT_(x):
    try:
        exec(x, globals())
    except:
        pass


CPython 3.6.9
IPython 7.16.1

numpy 1.18.5
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed
pytorch_lightning 1.2.0


In [9]:
###
### Common ###
###

import os, io, time, random, math, base64
import json, requests, sys

_IMPORT_('import numpy as np')
_IMPORT_('import pandas as pd')
_IMPORT_('from tqdm.notebook import tqdm')

###
### Torch ###
###

_IMPORT_('import torch')
_IMPORT_('import torch.nn as nn')
_IMPORT_('import torch.nn.functional as F')
_IMPORT_('import torch.optim as O')
_IMPORT_('from torchvision import models as M')
_IMPORT_('from torchvision import transforms as T')
_IMPORT_('from torch.utils.data import Dataset, DataLoader')

###
### Display ###
###

_IMPORT_('import cv2')
_IMPORT_('from PIL import Image')
_IMPORT_('from torchvision.utils import make_grid')
_IMPORT_('import matplotlib.pyplot as plt')
_IMPORT_('import plotly')
_IMPORT_('import plotly.graph_objects as go')

# plotly.offline.init_notebook_mode(connected=False)

def show_video(video_path, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    mp4 = open(video_path, 'rb').read()
    data_url = 'data:video/mp4;base64,' + base64.b64encode(mp4).decode()
    return HTML('<video %s %s controls src="%s" type="video/mp4"/>' % (W, H, data_url))

def show_image(image_path, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    img = open(image_path, 'rb').read()
    data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

from k12libs.utils.nb_easy import RACEURL

In [3]:
API_INFERENCE = f'{RACEURL}/raceai/framework/inference'
API_POPMSG = f'{RACEURL}/raceai/private/popmsg'
MODEL_TASK = 'zmq.repnet_tf.inference'
MSGKEY = 'zmq.repnet_tf.test'
TEST_VIDEO = 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet_test.mp4'
API_INFERENCE, API_POPMSG 

('http://116.85.5.40:9119/raceai/framework/inference',
 'http://116.85.5.40:9119/raceai/private/popmsg')

## 接口测试

### 输入

```json

{
    "task": "zmq.repnet_tf.inference",  // 任务类型, 不可修改
    "cfg": {
        "pigeon": {  // 信鸽, 由服务器自行添加字段, 异步数据输出时会携带这些数据返回, 其中"msgkey", "user_code"必须有
            "msgkey": "zmq.repnet_tf.test",  // 异步数据Redis消息队列key
            "user_code": "123", // 任务由谁触发
        },
        "video": "https://raceai.s3.didiyunapi.com/data/media/videos/repnet_test.mp4", // 视频地址
        "save_video": True,   // 是否保存处理后的视频 (开启后, 速度非常慢)
        "batch_size": 20,     // batch_size
        "threshold": 0.2,     // 模型预测分数阀值
        "in_threshold": 0.5,  // 模型预测帧是否在周期内的阀值
        "strides": [1, 2, 3, 4],     // 送入模型的帧跳跃步长, 1表示逐个帧, 2表示跳跃一帧..., 模型会自行选择步长最合适的
        "constant_speed": False,     // 视频周期运动速度是否常数
        "median_filter": True,       // 是否对周期检测做中值滤波
        "fully_periodic": False      // 周期检测是否使用全周期(所有帧), 而不是由in_threshold指定的阀值过滤
    }
}
```

In [11]:
cfgdata = '''{
    "task": "%s",
    "cfg": {
        "pigeon": {
            "msgkey": "%s",
            "user_code": "123",
        },
        "video": "%s",
        "save_video": True,
        "batch_size": 20,
        "threshold": 0.2,
        "in_threshold": 0.5,
        "strides": [1, 2, 3, 4],
        "constant_speed": False,
        "median_filter": True,
        "fully_periodic": False
    }
}''' % (MODEL_TASK, MSGKEY, TEST_VIDEO)

print("Input:\n\n", cfgdata)

Input:

 {
    "task": "zmq.repnet_tf.inference",
    "cfg": {
        "pigeon": {
            "msgkey": "zmq.repnet_tf.test",
            "user_code": "123",
        },
        "video": "https://raceai.s3.didiyunapi.com/data/media/videos/repnet_test.mp4",
        "batch_size": 20,
        "save_video": True,
        "threshold": 0.2,
        "in_threshold": 0.5,
        "strides": [1, 2, 3, 4],
        "constant_speed": False,
        "median_filter": True,
        "fully_periodic": False
    }
}


### 输出

```json
 {
    "pigeon": {
        "msgkey": "zmq.repnet_tf.test",
        "user_code": "123"
    },
    "task": "zmq.repnet_tf.inference",
    "errno": 0,
    "progress": 100.0,
    "target_mp4": "https://raceai.s3.didiyunapi.com/outputs/123/repnet_tf-target_40.mp4",
    "target_json": "https://raceai.s3.didiyunapi.com/outputs/123/repnet_tf-results.json"
}
```

In [23]:
reqdata = eval(cfgdata)
json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
json.loads(requests.post(url=API_INFERENCE, json=reqdata).text)

{'errno': 0}

In [24]:
for _ in range(200):
    result = json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
    if len(result) > 0 and 'progress' in result[-1]:
        progress = int(result[-1]['progress'])
        print("\r", end="")
        print("Progress: {}%: ".format(progress), "▋" * (progress // 2), end="")
        sys.stdout.flush()
        if progress >= 100.0:
            print('\nOutput:\n\n', json.dumps(result[-1], indent=4))
            break
    time.sleep(3)

Progress: 100%:  ▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋
Output:

 {
    "pigeon": {
        "msgkey": "zmq.repnet_tf.test",
        "user_code": "123"
    },
    "task": "zmq.repnet_tf.inference",
    "errno": 0,
    "progress": 100.0,
    "target_mp4": "https://raceai.s3.didiyunapi.com/outputs/123/repnet_tf-target_40.mp4",
    "target_json": "https://raceai.s3.didiyunapi.com/outputs/123/repnet_tf-results.json"
}


## JSON结果数据

```json
{
    "period": 58.022187843507346,      // 视频周期平均值(帧数)
    "score": 0.6532081365585327,       // 视频预测的平均分数
    "stride": 2,                       // 最佳步长
    "fps": 29.916666666666668,         // 视频帧率
    "num_frames": 2307,                // 视频帧总数
    "infer_time": 7.5414674282073975,  // 模型推理用时
    "mkvideo_time": 88.88              // 处理合成视频用时
    "frames_period": [                 // 每一帧的周期信息
        { 
            "image_id": "0.jpg",                         // 第一帧
            "within_period": 0.29910561442375183,        // 属于周期内的帧的置信度
            "pframe_counts": 0.0                         // 每一帧所代表周期数量 (小数)
        },                                           
        {
            "image_id": "1.jpg",                         // 第二帧
            "within_period": 0.29910561442375183,
            "pframe_counts": 0.0
        },
        {
            "image_id": "2.jpg",
            "within_period": 0.31776148080825806,
            "pframe_counts": 0.0
        },
        {
            "image_id": "3.jpg",
            "within_period": 0.31776148080825806,
            "pframe_counts": 0.0
        },
      ....
    ]
}
```

## References